<a href="https://colab.research.google.com/github/Aafreen2603/deep-learning/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np

### Explore the RNN type

In [ ]:
# set layer parameters
input_size = 9 # no. of features to extract (e.g. no of data channels)
hidden_size = 16 # no. of units in the hidden state
num_layers = 1 # no. of vertical stacks in hidden layers (note: only final layer gives an output)
actfun = 'tanh'
bias = True

In [ ]:
# create an RNN instance
rnn = nn.RNN(input_size,hidden_size,num_layers,nonlinearity=actfun,bias=bias)
print(rnn)

RNN(9, 16)


In [ ]:
# check out the source code for more detailed info about this class
nn.RNN

torch.nn.modules.rnn.RNN

In [ ]:
# set data parameters
seqlength = 5
batchsize = 2

In [ ]:
# create some data
X = torch.rand(seqlength,batchsize,input_size)

In [ ]:
# create a hidden layer (typically initialized as zeros)
hidden = torch.zeros(num_layers,batchsize,hidden_size)

In [ ]:
# run some data through the model and show the output sizes
y,h = rnn(X,hidden)
print(f' Input shape: {list(X.shape)}')
print(f'Hidden shape: {list(h.shape)}')
print(f'Output shape: {list(y.shape)}')

 Input shape: [5, 2, 9]
Hidden shape: [1, 2, 16]
Output shape: [5, 2, 16]


In [ ]:
## Default hidden state is all zeros if nothing specified:
y,h1 = rnn(X,hidden)
print(h1), print('\n\n')

tensor([[[ 0.0969,  0.4915,  0.2633, -0.1885, -0.3025, -0.1939,  0.1583,
           0.3139,  0.2542,  0.5571,  0.5691,  0.2618,  0.3612,  0.0586,
          -0.4617,  0.1753],
         [ 0.4404,  0.3500,  0.2555,  0.1649, -0.4627,  0.2182, -0.1873,
           0.2826, -0.0714,  0.6613,  0.2401, -0.3291,  0.5788,  0.2645,
          -0.4951, -0.1928]]], grad_fn=<StackBackward0>)





(None, None)

In [ ]:
y,h2 = rnn(X)
print(h2), print('\n\n')

tensor([[[ 0.0969,  0.4915,  0.2633, -0.1885, -0.3025, -0.1939,  0.1583,
           0.3139,  0.2542,  0.5571,  0.5691,  0.2618,  0.3612,  0.0586,
          -0.4617,  0.1753],
         [ 0.4404,  0.3500,  0.2555,  0.1649, -0.4627,  0.2182, -0.1873,
           0.2826, -0.0714,  0.6613,  0.2401, -0.3291,  0.5788,  0.2645,
          -0.4951, -0.1928]]], grad_fn=<StackBackward0>)





(None, None)

In [ ]:
# they're the same! (meaning default=zeros)
print(h1-h2)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<SubBackward0>)


In [ ]:
# Check out the learned parameters and their sizes
for p in rnn.named_parameters():
  if 'weight' in p[0]:
    print(f'{p[0]} has size {list(p[1].shape)}')

weight_ih_l0 has size [16, 9]
weight_hh_l0 has size [16, 16]


Create a DL model class

In [ ]:
class RNNnet(nn.Module):
  def __init__(self,input_size,num_hidden,num_layers):
    super().__init__()

    # store parameters
    self.input_size = input_size
    self.num_hidden = num_hidden
    self.num_layers = num_layers

    # RNN Layer
    self.rnn = nn.RNN(input_size,num_hidden,num_layers)

    # linear layer for output
    self.out = nn.Linear(num_hidden,1)
  
  def forward(self,x):
    print(f'Input: {list(x.shape)}')

    # initialize hidden state for first input
    hidden = torch.zeros(self.num_layers,batchsize,self.num_hidden)
    print(f'Hidden: {list(hidden.shape)}')

    # run through the RNN layer
    y,hidden = self.rnn(x,hidden)
    print(f'RNN-out: {list(y.shape)}')
    print(f'RNN-hidden: {list(hidden.shape)}')
    
    # pass the RNN output through the linear output layer
    o = self.out(y)
    print(f'Output: {list(o.shape)}')
    return o,hidden

In [ ]:
# create an instance of the model and inspect
net = RNNnet(input_size,hidden_size,num_layers)
print(net), print(' ')

RNNnet(
  (rnn): RNN(9, 16)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
 


(None, None)

In [ ]:
# and check out all learnable parameters
for p in net.named_parameters():
  print(f'{p[0]} has size {list(p[1].shape)}')

rnn.weight_ih_l0 has size [16, 9]
rnn.weight_hh_l0 has size [16, 16]
rnn.bias_ih_l0 has size [16]
rnn.bias_hh_l0 has size [16]
out.weight has size [1, 16]
out.bias has size [1]


In [ ]:
# test the model with some data
# create some data
X = torch.rand(seqlength,batchsize,input_size)
y = torch.rand(seqlength,batchsize,1)
yHat,h = net(X)

Input: [5, 2, 9]
Hidden: [1, 2, 16]
RNN-out: [5, 2, 16]
RNN-hidden: [1, 2, 16]
Output: [5, 2, 1]


In [ ]:
# try a loss function
lossfun = nn.MSELoss()
lossfun(yHat,y)

tensor(0.7185, grad_fn=<MseLossBackward0>)